In [8]:
import pandas as pd

df1 = pd.read_excel('csv/DSF1.xlsx')
df2 = pd.read_excel('csv/DSF2.xlsx')
df3 = pd.read_excel('csv/DSF3.xlsx')

col_list = ['id', 'IP', 'dt_completion', 't_used', 'mail', 'name']

for i, col in enumerate(df2.columns):
    if col not in col_list:
        df2.rename(columns={col: f'column_{i+1}'}, inplace=True)

for i, col in enumerate(df3.columns):
    if col not in col_list:
        df3.rename(columns={col: f'column_{i+1}'}, inplace=True)

merged_df = pd.merge(df2, df3, on=list(df2.columns) + list(df3.columns),  how='outer')

In [9]:
columns_to_delete = [col for col in merged_df.columns if col.startswith('column_') and int(col.split('_')[1]) % 3 != 0]

In [10]:
merged_df = merged_df.drop(columns=columns_to_delete, axis=1)

In [11]:
cols_to_process = merged_df.columns[6:]

merged_df[cols_to_process] = merged_df[cols_to_process].replace(regex={r'1 из 1': 1, '0 из 1': 0})
merged_df[cols_to_process] = merged_df[cols_to_process].fillna(0).astype(int)

df = merged_df.copy()

In [12]:
s_questions = [10, 9, 8, 9, 8, 8, 8, 9, 8, 8, 8, 4]

start_col = 6
end_col = start_col

for i, num_questions in enumerate(s_questions):
    end_col += num_questions
    section_num = i + 1
    section_name = 'S' + str(section_num)
    section_cols = list(range(start_col, end_col))
    df[str(section_name)] = df.iloc[:, section_cols].sum(axis=1)
    start_col = end_col

columns_to_del = [col for col in merged_df.columns if col.startswith('column_')]
df = df.drop(columns=columns_to_del, axis=1)

In [13]:
df = pd.merge(df, df1, on=list(df.columns) + list(df1.columns),  how='outer')
df['t_used'] = df['t_used'].astype(str)
df['t_used'] = pd.to_timedelta(df['t_used'])
df = df[df['t_used'] >= pd.Timedelta(minutes=20)]

In [14]:
df.sort_values('dt_completion', inplace=True)

df = df.groupby('IP').first().reset_index()

In [15]:
df = df.groupby('mail').first().reset_index()

In [16]:
df = df.groupby('name').first().reset_index()

In [18]:
df.index.name = 'index'

In [19]:
private = df.copy(['name', 'mail', 'IP', 'dt_completion', 't_used'])
df = df.drop(['name', 'mail', 'IP', 'dt_completion', 't_used'], axis=1)

In [20]:
df

,id,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12
index,,,,,,,,,,,,,
0,170287232,2,3,0,1,2,2,1,2,2,1,1,1
1,171738755,5,4,1,4,2,3,3,0,4,3,2,1
2,123082919,3,3,2,0,1,3,3,2,2,2,1,2
3,122846297,3,5,2,2,3,2,4,2,5,4,3,1
4,123039223,4,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,121722659,5,5,0,4,3,5,4,4,2,2,3,4
377,122771403,4,1,3,3,4,3,3,4,5,3,2,1
378,193988203,3,3,3,3,3,3,4,3,5,3,3,3
